Fine-Tuning a Hugging Face Model for Customer Support or Organizational Use Case


**Objective**: In this assignment, you will fine-tune a pre-trained Hugging Face model on a dataset related to customer support or organizational use cases. The goal is to enhance the model's performance for real-world applications in customer interaction or business operations. You are encouraged to explore techniques such as LoRA, DPO, and the Unsloth library to optimize your model for better efficiency and accuracy.

In [1]:
# -----------------------------
# Hugging Face Transformers & Datasets
# -----------------------------
!pip install --upgrade --no-cache-dir transformers==4.34.0 datasets

# -----------------------------
# PEFT (Parameter-Efficient Fine-Tuning)
# -----------------------------
!pip install --upgrade --no-cache-dir peft

# -----------------------------
# TRL (SFTTrainer / TRLTrainer)
# -----------------------------
!pip install --upgrade --no-cache-dir trl

# -----------------------------
# PyTorch + torchvision + torchaudio (CUDA 11.8 for T4)
# -----------------------------
!pip install --upgrade --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# -----------------------------
# Accelerate (optional, recommended for multi-GPU / memory optimization)
# -----------------------------
!pip install --upgrade --no-cache-dir accelerate

# -----------------------------
# Notes
# - JSON, logging, and built-in modules are already included with Python
# - Restart the Kaggle kernel after installation to ensure correct versions are loaded
# -----------------------------


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 6.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 127.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 359.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 336.3 

In [3]:
########################
### QWEN FINE-TUNING ###
########################
import json, sys
import logging
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

# ------------------- Logging Setup -------------------
logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%H:%M:%S"
)
logger = logging.getLogger(__name__)
logger.info("Logger initialized.")

# ------------------- Device -------------------
if torch.cuda.is_available():
    device = "cuda"
    n_gpus = torch.cuda.device_count()
    logger.info(f"CUDA is available. Using {n_gpus} GPU(s).")
else:
    device = "cpu"
    n_gpus = 0
    logger.info("CUDA not available. Using CPU.")

# ------------------- Load Model & Tokenizer -------------------
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
logger.info(f"Loading model: {model_name}")
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map="auto")

# # Wrap model for multi-GPU if available
# if n_gpus > 1:
#     logger.info("Wrapping model with DataParallel for multi-GPU training.")
#     model = torch.nn.DataParallel(model)

# model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, device_map="auto")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
logger.info("Model and tokenizer loaded successfully.")

print("Model and tokenizer loaded successfully.\n")

# ------------------- Load JSONL Data -------------------
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            data.append(json.loads(line))
    logger.info(f"Loaded {len(data)} examples from {file_path}")
    return data

train_data = load_jsonl('/kaggle/input/fitness-faq-dataset/qa_fitness_nutrition_train.jsonl')
eval_data = load_jsonl('/kaggle/input/fitness-faq-dataset/qa_fitness_nutrition_val.jsonl')

train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)
logger.info("Datasets created.")

print("Datasets created.\n")

# ------------------- Tokenization -------------------
def preprocess_function(example):
    question = example.get("question", "")
    answer = example.get("answer", "")
    prompt = f"User: {question}\nAssistant: {answer}"
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

logger.info("Tokenizing train dataset...")
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=False)
logger.info("Tokenizing eval dataset...")
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=False)
print("Tokenizing is done...\n")

# ------------------- LoRA Config -------------------
lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
logger.info("LoRA configuration set.")
print("LoRA Configuration is configured...\n")

# ------------------- Training Arguments -------------------
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,  # effective batch size 32
    learning_rate=5e-5,
    warmup_steps=40,                 # 5% of total steps
    num_train_epochs=4,              # 4–5 epochs
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    output_dir="./qwen2.5-0.5b-finetuned",
    logging_dir="./logs",
    report_to="none"                 # disables wandb
)



# ------------------- SFT Trainer -------------------
logger.info("Initializing SFTTrainer...")
print("Initializing SFTTrainer...\n")
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    peft_config=lora_config,
    args=training_args
)
print("Initialized SFTTrainer...\n")

logger.info("Starting training...")
print("Training is starting...\n")
trainer.train()
logger.info("Training completed.")
print("Training completed...\n")

# ------------------- Save Model & Tokenizer -------------------
logger.info("Saving LoRA adapter weights...")
print("Saving LoRA adapter weights & tokenizer...\n")
trainer.model.module.save_pretrained("./qwen2.5-0.5b-finetuned") if n_gpus > 1 else trainer.model.save_pretrained("./qwen2.5-0.5b-finetuned")
logger.info("Saving tokenizer...")
tokenizer.save_pretrained("./qwen2.5-0.5b-finetuned")
logger.info("All artifacts saved successfully.")
print("Things are saved...\n")


Model and tokenizer loaded successfully.

Datasets created.



Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Tokenizing is done...

LoRA Configuration is configured...

Initializing SFTTrainer...



Truncating train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Initialized SFTTrainer...

Training is starting...



Step,Training Loss
10,13.473300
20,13.023200
30,11.988600
40,11.082600
50,8.888000
60,6.419000
70,3.716800
80,1.412300
90,0.804600
100,0.630700


Training completed...

Saving LoRA adapter weights & tokenizer...



AttributeError: 'Qwen2ForCausalLM' object has no attribute 'module'

In [4]:
logger.info("Saving LoRA adapter weights...")
print("Saving LoRA adapter weights & tokenizer...\n")

# Save PEFT adapter weights only
trainer.model.save_pretrained("./qwen2.5-0.5b-finetuned")

logger.info("Saving tokenizer...")
tokenizer.save_pretrained("./qwen2.5-0.5b-finetuned")

logger.info("All artifacts saved successfully.")


Saving LoRA adapter weights & tokenizer...



In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

# ------------------ Config ------------------
base_model_name = "Qwen/Qwen2.5-0.5B"
lora_model_path = "/kaggle/working/qwen2.5-0.5b-finetuned"  # Path to your LoRA adapter

# ------------------ Load Base Model ------------------
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")

# ------------------ Load LoRA Adapter ------------------
model = PeftModel.from_pretrained(model, lora_model_path)

# ------------------ Inference ------------------
faq_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

question = "What are the benefits of regular exercise?"
response = faq_pipeline(question, max_new_tokens=200, temperature=0.2)
print(response[0]['generated_text'])


Device set to use cuda:0


What are the benefits of regular exercise? Regular exercise can have many benefits for your health, including:

1. Weight loss: Exercise can help you lose weight by increasing your metabolism and burning calories.

2. Improved cardiovascular health: Exercise can help you lower your risk of heart disease and stroke.

3. Increased strength and endurance: Exercise can help you build muscle mass and increase your overall strength and endurance.

4. Improved balance and coordination: Exercise can help you maintain your balance and coordination, which can be important for activities such as walking, dancing, and playing sports.

5. Reduced risk of injury: Exercise can help you avoid injuries and improve your overall health.

6. Increased bone density: Exercise can help you build and maintain bone density, which can be important for preventing osteoporosis and other bone-related conditions.

7. Improved mood and energy: Exercise can help you feel better physically and mentally, which can impr

In [8]:
# MERGED with base model
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel

# Paths
base_model_name = "Qwen/Qwen2.5-0.5B"
adapter_path = "./qwen2.5-0.5b-finetuned"
merged_model_path = "./qwen2.5-0.5b-merged"

# Load base model
print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")

# Apply LoRA adapter
print("Applying LoRA adapter...")
model = PeftModel.from_pretrained(model, adapter_path)

# Merge LoRA into base model
print("Merging LoRA weights...")
model = model.merge_and_unload()

# Save merged model
print("Saving merged model...")
model.save_pretrained(merged_model_path)

# Load tokenizer from adapter path (or base)
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

# Create pipeline with merged model
faq_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example FAQ inference
question = "What are the benefits of strength training?"
response = faq_pipeline(question, max_new_tokens=100, temperature=0.7, top_p=0.9)[0]['generated_text']

print("\nQuestion:", question)
print("Answer:", response)


Loading base model...
Applying LoRA adapter...
Merging LoRA weights...
Saving merged model...


Device set to use cuda:0



Question: What are the benefits of strength training?
Answer: What are the benefits of strength training? Benefits of Strength Training

Benefits of Strength Training

Benefits of Strength Training

The human body is designed to adapt to changes in its environment. To stay fit, it is necessary to modify the way we train. There are certain factors that determine the effectiveness of strength training. If you want to improve your fitness level and lose weight, you must take into account certain things. These things include a proper diet, regular exercise, and proper form. Strength training can help you achieve your goals and improve your health
